<a href="https://colab.research.google.com/github/dayaYanez/MLTSA25_DYanez/blob/main/MLTSA_Lag_Llama_Fine_Tuning_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lag-Llama Fine Tuning MLTSA

## Prepare the repository

We first clone and install the required packages from the [GitHub repository](https://github.com/time-series-foundation-models/lag-llama/) that has the Lag-Llama architecture.

In [1]:
!git clone -b update-gluonts https://github.com/time-series-foundation-models/lag-llama/

Cloning into 'lag-llama'...
remote: Enumerating objects: 505, done.
remote: Counting objects: 100% (193/193), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 505 (delta 154), reused 109 (delta 109), pack-reused 312 (from 3)
Receiving objects: 100% (505/505), 283.34 KiB | 9.77 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [2]:
cd /content/lag-llama

/content/lag-llama


In [3]:
!pip uninstall jax jaxlib tensorboard -y

Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1
Found existing installation: tensorboard 2.18.0
Uninstalling tensorboard-2.18.0:
  Successfully uninstalled tensorboard-2.18.0


In [4]:
!pip install -U -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

We then download our pretrained model weights from [HuggingFace](https://huggingface.co/time-series-foundation-models/Lag-Llama) 🤗

In [5]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama

lag-llama.ckpt: 100% 29.5M/29.5M [00:00<00:00, 192MB/s]
Download complete. Moving file to /content/lag-llama/lag-llama.ckpt
/content/lag-llama/lag-llama.ckpt


## Imports

We import the required packages and the lag llama estimator object which we can use to make predictions.

In [6]:
from itertools import islice

import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm
import glob
import numpy as np

from scipy.signal import butter, lfilter

import torch

from lag_llama.gluon.estimator import LagLlamaEstimator

import sys
from types import ModuleType

from gluonts.evaluation import make_evaluation_predictions, MultivariateEvaluator
from gluonts.evaluation import Evaluator


<ipython-input-6-b6a62eef4d70>:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
# Create dummy module hierarchy
def create_dummy_module(module_path):
    """
    Create a dummy module hierarchy for the given path.
    Returns the leaf module.
    """
    parts = module_path.split('.')
    current = ''
    parent = None

    for part in parts:
        current = current + '.' + part if current else part
        if current not in sys.modules:
            module = ModuleType(current)
            sys.modules[current] = module
            if parent:
                setattr(sys.modules[parent], part, module)
        parent = current

    return sys.modules[module_path]

# Create the dummy gluonts module hierarchy
gluonts_module = create_dummy_module('gluonts.torch.modules.loss')

# Create dummy classes for the specific loss functions
class DistributionLoss:
    def __init__(self, *args, **kwargs):
        pass

    def __call__(self, *args, **kwargs):
        return 0.0

    def __getattr__(self, name):
        return lambda *args, **kwargs: None

class NegativeLogLikelihood:
    def __init__(self, *args, **kwargs):
        pass

    def __call__(self, *args, **kwargs):
        return 0.0

    def __getattr__(self, name):
        return lambda *args, **kwargs: None

# Add the specific classes to the module
gluonts_module.DistributionLoss = DistributionLoss
gluonts_module.NegativeLogLikelihood = NegativeLogLikelihood

We create a function for Lag-Llama inference that we can reuse. This function returns the predictions for the given prediction horizon. The forecast will be of shape (num_samples, prediction_length), where `num_samples` is the number of samples sampled from the predicted probability distribution for each timestep.

In [8]:
def get_lag_llama_predictions(dataset, prediction_length, context_length=32,
              num_samples=20, device="cuda", batch_size=64, nonnegative_pred_samples=True):
    ckpt = torch.load("lag-llama.ckpt", map_location=device, weights_only=False) # modified
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],

        nonnegative_pred_samples=nonnegative_pred_samples,

        # linear positional encoding scaling
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },

        batch_size=batch_size,
        num_parallel_samples=num_samples,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))

    return forecasts, tss

# Zero Shot

## data : `M4-Weekly`

In [ ]:
from gluonts.dataset.repository import get_dataset
dataset = get_dataset("m4_weekly")
train_dataset = dataset.train
test_dataset = dataset.test

In [ ]:
print(f"dataset length: {len(test_dataset)}")

dataset length: 359


In [ ]:
prediction_length = dataset.metadata.prediction_length
context_length = prediction_length * 3
num_samples = 20
device = "cuda"
print(f"prediction_length {prediction_length}")

prediction_length 13


## NEW DATASET

In [ ]:
# @title
!git clone https://github.com/nlandolfi/acc2022treelinearcascades_stocks.git

In [ ]:
# @title
files = glob.glob("acc2022treelinearcascades_stocks/data/*")
files

In [ ]:
# @title
pd.read_csv(files[0], index_col=0)

In [ ]:
# @title
data = np.zeros((len(files)-1,  pd.read_csv(files[0], index_col=0).shape[0])) #prep data array for all files

In [ ]:
# @title
j = 0
for i,f in enumerate(files):
  _ = pd.read_csv(f, index_col=0)
  if len(_) < data.shape[1]:
    continue

  data[j, :len(_[' Volume'])] = _[' Volume'].values
  j += 1
data.shape

In [ ]:
# @title
x = pd.to_datetime(_.index)
fig, ax = plt.subplots(3,3, figsize=(20,10), sharex=True)
for i in range(9):
  ax[i%3, i//3].plot(x, data[i])
  if i%3 > 1:
    ax[i%3, i//3].set_xticklabels(labels=ax[i%3, i//3].get_xticklabels(), rotation=45)
plt.show()


In [ ]:
# @title
x.diff()

In [ ]:
# @title
df = pd.DataFrame(data=data).T
df.index= pd.to_datetime(x, format="%m/%d/%y")
df_resampled = df.resample('2d').mean()#.fillna(df.mean())
df_resampled = df_resampled.interpolate(limit_direction='both')
df_resampled.index.diff().unique()

In [ ]:
# @title
df_resampled.shape

In [ ]:
# @title
x = df_resampled.index
fig, ax = plt.subplots(3,3, figsize=(20,10), sharex=True)
for i in range(9):
  ax[i%3, i//3].plot(x, df_resampled.iloc[:,i])
  if i%3 > 1:
    ax[i%3, i//3].set_xticklabels(labels=ax[i%3, i//3].get_xticklabels(), rotation=45)
plt.show()


In [ ]:
# @title
# prompt: lowpass filter the imput data
from scipy.signal import butter, lfilter, filtfilt


def butter_lowpass(cutoff, fs, order=5):
    """
    Design a digital Butterworth low-pass filter.

    The Butterworth filter is characterized by a maximally flat magnitude response
    in the passband and a monotonic roll-off in the stopband.  This function
    calculates the filter coefficients for a low-pass Butterworth filter.

    Args:
        cutoff (float): The cutoff frequency of the filter in Hz.  This is the
            frequency at which the filter's gain is reduced by 3 dB (half-power).
        fs (float): The sampling rate of the signal to be filtered, in Hz.
        order (int, optional): The order of the filter.  Higher order filters
            provide a steeper roll-off but also increase the filter's complexity
            and introduce more phase delay.  The default is 5.

    Returns:
        tuple: A tuple of two NumPy arrays, (b, a).
            - b (ndarray): The numerator coefficients of the filter's transfer function.
            - a (ndarray): The denominator coefficients of the filter's transfer function.

    The filter's transfer function is defined as:
    ∣H(jω)∣^2 = 1 / (1 + (ω/ωc)^(2n))

    Where:
    - ω is the angular frequency (2 * pi * frequency)
    - ωc is the cutoff angular frequency (2 * pi * cutoff)
    - n is the filter order
    """

    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    """
    Apply a digital Butterworth low-pass filter to input data.

    This function designs and applies a Butterworth low-pass filter to the given
    input data using forward-backward filtering to minimize phase distortion.

    Args:
        data (ndarray): The input data to be filtered.  It should be a 1-D NumPy array.
        cutoff (float): The cutoff frequency of the filter in Hz.
            Frequencies above this will be attenuated.
        fs (float): The sampling rate of the input data, in Hz.
        order (int, optional): The order of the filter. Higher order filters
            provide a steeper roll-off but can increase complexity. The default is 5.

    Returns:
        ndarray: The filtered data, with the same shape as the input data.

    """
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

fs = 1/48  # Replace with your actual sampling frequency this is in hours

# Choose cutoff frequency (adjust based on desired filtering characteristics)
cutoff = 1/48/50

# Apply filter to each time series in 'data'
filtered_data = np.zeros_like(df_resampled.T.values)
for i in range(filtered_data.shape[0]):
    filtered_data[i, :] = butter_lowpass_filter(df_resampled.iloc[:,i], cutoff, fs, order=5)

# Now 'filtered_data' contains the low-pass filtered time series data
# You can proceed to plot or further analyze the filtered data

fig, ax = plt.subplots(3,3, figsize=(20,10), sharex=True)
for i in range(9):
  ax[i%3, i//3].plot(x, df_resampled.iloc[:,i])
  ax[i%3, i//3].plot(x, filtered_data[i], 'r')
  if i%3 > 1:
    ax[i%3, i//3].set_xticklabels(labels=ax[i%3, i//3].get_xticklabels(), rotation=45)
plt.show()


In [ ]:
# @title
df_filtered = pd.DataFrame(data=filtered_data).T
df_filtered.index = x
df_filtered.index.diff().unique(),


In [ ]:
# @title
df_filtered

In [ ]:
# @title
#we are not going to use it but useful cause it may be inputts in gluonts: how do i make it a single column df?
df_filtered_long = df_filtered.reset_index()
df_filtered_long = df_filtered_long.melt(id_vars="Date")
df_filtered_long.set_index("Date", inplace=True)
df_filtered_long.index = pd.to_datetime(df_filtered_long.index)
df_filtered_long

In [ ]:
# @title
ax = df_filtered.plot(legend=False)
ax.set_xticklabels(labels=ax.get_xticklabels(), rotation=90);


In [ ]:
# @title
df_filtered.isna().sum().sum()

In [ ]:
# @title
#import into gluonts
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset

start_date = df_filtered.index[0]
train_data = []
test_data = []
for i in range(len(df_filtered.columns)):
    # Create synthetic time series with some dependencies
    target = df_filtered.iloc[:,i]

    train_target = target[:-prediction_length]
    test_target = target

    train_data.append({
        FieldName.TARGET: train_target,
        FieldName.START: start_date,
        FieldName.ITEM_ID: f"series_{i}"
    })
    test_data.append({
        FieldName.TARGET: test_target,
        FieldName.START: start_date,
        FieldName.ITEM_ID: f"series_{i}"
    })

# Convert to GluonTS ListDataset
train_dataset = ListDataset(train_data, freq="2d")
test_dataset = ListDataset(test_data, freq="2d")

## predict

In [ ]:
list(train_dataset)[0]

In [ ]:
plt.plot(list(train_dataset)[0]["target"])


In [ ]:
forecasts, tss = get_lag_llama_predictions(
    test_dataset,
    prediction_length=prediction_length,
    num_samples=num_samples,
    context_length=context_length,
    device=device
)

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

# Iterate through the first 9 series, and plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-25 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

# Fine-tuning

Let us fine-tune the Lag-Llama base model with a few data-specific changes. Feel Free to change the hyperparameters below:

In [ ]:
ckpt = torch.load("lag-llama.ckpt", map_location=device, weights_only=False)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # distr_output="neg_bin",
        # scaling="mean",
        nonnegative_pred_samples=True,
        aug_prob=0,
        lr=5e-4,
        #lr=5e-6,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        time_feat=estimator_args["time_feat"],

        # rope_scaling={
        #     "type": "linear",
        #     "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        # },

        batch_size=64,
        num_parallel_samples=num_samples,
        trainer_kwargs = {"max_epochs": 50,}, # <- lightning trainer arguments
    )

In [ ]:
predictor = estimator.train(train_dataset,
    cache_data=True, shuffle_buffer_length=1000)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
        test_dataset,
        predictor=predictor,
        num_samples=num_samples
    )

In [ ]:
forecasts = list(tqdm(forecast_it, total=len(test_dataset), desc="Forecasting batches"))

In [ ]:
tss = list(tqdm(ts_it, total=len(test_dataset), desc="Ground truth"))

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

# Iterate through the first 9 series, and plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-25 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
evaluator = Evaluator()
agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))

In [ ]:
agg_metrics